In [1]:
import cv2
import easyocr
import numpy as np
from PIL import ImageGrab, Image

In [5]:
import time

In [15]:
import cv2
import easyocr
import numpy as np
from PIL import ImageGrab, Image
import time

def take_screenshot_of_comment_section(file_name, keywords, platform="generic"):
    """
    Toma un pantallazo de la sección de comentarios en la pantalla basada en la detección de palabras clave usando EasyOCR.
    La captura se toma desde la ubicación de la palabra clave hacia abajo y se extiende horizontalmente para incluir los comentarios.

    :param file_name: Nombre del archivo donde se guardará el pantallazo.
    :param keywords: Lista de palabras clave para identificar la sección de comentarios.
    :param platform: Plataforma específica en la que se está trabajando.
    """
    # Espera 3 segundos antes de tomar el pantallazo
    print("Esperando 3 segundos antes de tomar el pantallazo...")
    time.sleep(3)

    # Captura toda la pantalla
    screenshot = ImageGrab.grab()
    screenshot_np = np.array(screenshot)

    # Convierte la imagen a escala de grises para el OCR
    gray_screenshot = cv2.cvtColor(screenshot_np, cv2.COLOR_BGR2GRAY)

    # Inicializa el lector de EasyOCR
    reader = easyocr.Reader(['en', 'es'])  # Ajusta los idiomas según necesites

    # Aplica OCR para detectar texto en la pantalla
    ocr_result = reader.readtext(gray_screenshot)

    screen_width = screenshot_np.shape[1]
    screen_height = screenshot_np.shape[0]

    # Variables para la mejor sección de comentarios
    best_section = None

    # Buscar la palabra clave en toda la pantalla
    for (bbox, text, prob) in ocr_result:
        for keyword in keywords:
            if keyword.lower() in text.lower():
                # Encuentra las coordenadas de la palabra clave
                (top_left, top_right, bottom_right, bottom_left) = bbox
                left = int(min(top_left[0], bottom_left[0]))
                top = int(min(top_left[1], top_right[1]))

                # Extiende la captura hacia abajo y horizontalmente
                new_bottom = screen_height  # Desde la palabra clave hasta la parte inferior de la pantalla
                new_left = max(left - 20, 0)
                new_right = min(screen_width, screen_width)  # Capturar todo el ancho desde la palabra clave

                # Define la región a capturar
                best_section = (new_left, top, new_right, new_bottom)
                break
        if best_section:
            break

    # Si se detectó una sección adecuada, tomar un pantallazo
    if best_section:
        best_left, best_top, best_right, best_bottom = best_section
        comment_section_screenshot = screenshot.crop((best_left, best_top, best_right, best_bottom))
        comment_section_screenshot.save(file_name)
        print(f"Pantallazo guardado como {file_name} en la plataforma {platform}.")
    else:
        print(f"No se detectó ninguna sección de comentarios en la plataforma: {platform}.")

# Ejemplo de uso
keywords = ["Chat", "comments", "comentarios", "chatting", "discussion", "live chat"]
platform = "twitch"  # Cambia por la plataforma que estés usando
take_screenshot_of_comment_section("comentarios.png", keywords, platform=platform)



Esperando 3 segundos antes de tomar el pantallazo...


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Pantallazo guardado como comentarios.png en la plataforma twitch.
